# Training a ML model using CICIoT2023

This notebook shows how a LogisticRegression model can be trained using the CICIoT2023 csv files.

In [1]:
#Regular EDA and plotting libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# We want our plots to appear in the notebook
%matplotlib inline

## Models
from tqdm import tqdm, trange
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

## Model evaluators
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from sklearn.metrics import RocCurveDisplay

import joblib


In [2]:
DATASET_DIRECTORY = 'CICIoT2023/'

### Importing Dataset

In [3]:
df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
df_sets.sort()
training_sets = df_sets[:int(len(df_sets)*.8)]
test_sets = df_sets[int(len(df_sets)*.8):]

In [4]:
X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
       'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count',
       'syn_count', 'fin_count', 'urg_count', 'rst_count',
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
       'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
       'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
       'Radius', 'Covariance', 'Variance', 'Weight',
]
y_column = 'label'

### Scaling

In [5]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()

In [8]:
for train_set in tqdm(training_sets):
    scaler.fit(pd.read_csv(DATASET_DIRECTORY + train_set)[X_columns])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 135/135 [06:35<00:00,  2.93s/it]


# Classification: 8 (7+1) classes

In [9]:
dict_7classes = {}
dict_7classes['DDoS-RSTFINFlood'] = 'DDoS'
dict_7classes['DDoS-PSHACK_Flood'] = 'DDoS'
dict_7classes['DDoS-SYN_Flood'] = 'DDoS'
dict_7classes['DDoS-UDP_Flood'] = 'DDoS'
dict_7classes['DDoS-TCP_Flood'] = 'DDoS'
dict_7classes['DDoS-ICMP_Flood'] = 'DDoS'
dict_7classes['DDoS-SynonymousIP_Flood'] = 'DDoS'
dict_7classes['DDoS-ACK_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-UDP_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-ICMP_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-SlowLoris'] = 'DDoS'
dict_7classes['DDoS-HTTP_Flood'] = 'DDoS'

dict_7classes['DoS-UDP_Flood'] = 'DoS'
dict_7classes['DoS-SYN_Flood'] = 'DoS'
dict_7classes['DoS-TCP_Flood'] = 'DoS'
dict_7classes['DoS-HTTP_Flood'] = 'DoS'


dict_7classes['Mirai-greeth_flood'] = 'Mirai'
dict_7classes['Mirai-greip_flood'] = 'Mirai'
dict_7classes['Mirai-udpplain'] = 'Mirai'

dict_7classes['Recon-PingSweep'] = 'Recon'
dict_7classes['Recon-OSScan'] = 'Recon'
dict_7classes['Recon-PortScan'] = 'Recon'
dict_7classes['VulnerabilityScan'] = 'Recon'
dict_7classes['Recon-HostDiscovery'] = 'Recon'

dict_7classes['DNS_Spoofing'] = 'Spoofing'
dict_7classes['MITM-ArpSpoofing'] = 'Spoofing'

dict_7classes['BenignTraffic'] = 'Benign'

dict_7classes['BrowserHijacking'] = 'Web'
dict_7classes['Backdoor_Malware'] = 'Web'
dict_7classes['XSS'] = 'Web'
dict_7classes['Uploading_Attack'] = 'Web'
dict_7classes['SqlInjection'] = 'Web'
dict_7classes['CommandInjection'] = 'Web'


dict_7classes['DictionaryBruteForce'] = 'BruteForce'

In [ ]:
# Define the models
ML_models = [
    LogisticRegression(n_jobs=-1),
    Perceptron(),
    AdaBoostClassifier(),
    RandomForestClassifier(n_jobs=-1),
    DecisionTreeClassifier()
]

# Define the model names
ML_names = [
    "LogisticRegression",
    "Perceptron",
    "AdaBoost",
    "RandomForest",
    "DecisionTree"
]

# Train each model on each dataset
for train_set in tqdm(training_sets):
    d = pd.read_csv(DATASET_DIRECTORY + train_set)
    d[X_columns] = scaler.transform(d[X_columns])

    # Convert the labels using the dictionary
    new_y = [dict_7classes[k] for k in d[y_column]]
    d[y_column] = new_y

    # Train the models
    for i, model in enumerate(ML_models):
        model.fit(d[X_columns], d[y_column])
        print(f"{ML_names[i]} has been trained on {train_set}")
        # Save the trained model
        filename = f"model_7classes_{ML_names[i]}.sav"
        joblib.dump(model, filename)
    del d


  0%|                                                                                                                                                 | 0/135 [00:00<?, ?it/s]/home/sc/0.DCU/00.Project/ML_based_IDS_for_IoT/project/env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression has been trained on part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Perceptron has been trained on part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
AdaBoost has been trained on part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
RandomForest has been trained on part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv


  1%|▉                                                                                                                                     | 1/135 [01:42<3:49:46, 102.89s/it]

DecisionTree has been trained on part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv


/home/sc/0.DCU/00.Project/ML_based_IDS_for_IoT/project/env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

LogisticRegression has been trained on part-00108-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Perceptron has been trained on part-00108-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
AdaBoost has been trained on part-00108-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
RandomForest has been trained on part-00108-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv


 81%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▍                         | 109/135 [3:06:09<35:35, 82.13s/it]

DecisionTree has been trained on part-00108-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv


/home/sc/0.DCU/00.Project/ML_based_IDS_for_IoT/project/env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression has been trained on part-00109-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Perceptron has been trained on part-00109-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv


In [ ]:
y_test = []
preds = {i:[] for i in range(len(ML_models))}
for test_set in tqdm(test_sets):
    d_test = pd.read_csv(DATASET_DIRECTORY + test_set)
    d_test[X_columns] = scaler.transform(d_test[X_columns])
    new_y = [dict_7classes[k] for k in d_test[y_column]]
    d_test[y_column] = new_y

    y_test += list(d_test[y_column].values)

    for i in range(len(ML_models)):
        model = ML_models[i]
        y_pred = list(model.predict(d_test[X_columns]))
        preds[i] = preds[i] + y_pred
    # Saving the predictions and y_test using joblib
    joblib.dump(preds, "predictions_data_7classes.sav")
    joblib.dump(y_test, "y_test_7classes.sav")

In [ ]:
# Initialize the metrics_store_7classes dictionary here
metrics_store_7classes = {}

for k, v in preds.items():
    y_pred = v
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    metrics_store_7classes[ML_names[k]] = {
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1": f1
    }
    
    print('accuracy_score: ', accuracy)
    print('recall_score: ', recall)
    print('precision_score: ', precision)
    print('f1_score: ', f1)
    print()
    print()
    print()
    
# Save the metrics_store using joblib
joblib.dump(metrics_store_7classes, "model_metrics_7_classes.sav")

In [ ]:
# Lists to store the evaluation metrics for each model
accuracies = []
recalls = []
precisions = []
f1_scores = []

# Calculate and store metrics for each model
for k, v in preds.items():
    y_pred = v
    accuracies.append(accuracy_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred, average='macro'))
    precisions.append(precision_score(y_test, y_pred, average='macro'))
    f1_scores.append(f1_score(y_test, y_pred, average='macro'))

# Set up the matplotlib figure
plt.figure(figsize=(15, 12))

# Define metrics and their names for plotting
metrics = [accuracies, recalls, precisions, f1_scores]
metric_names = ['Accuracy', 'Recall', 'Precision', 'F1-Score']

# Plot each metric
for i, (metric, metric_name) in enumerate(zip(metrics, metric_names)):
    plt.subplot(2, 2, i+1)
    sns.barplot(x=ML_names, y=metric, palette="viridis")
    plt.title(f'Model Comparison: {metric_name}')
    plt.ylabel(metric_name)
    plt.xticks(rotation=45)
    plt.ylim(0, 1)  # Since all metrics are in the range [0, 1]

plt.tight_layout()
plt.show()
